## 來玩看看VGG

### 匯入會用到的模組

In [1]:
#匯入numpy(處理數值)
import numpy as np

#匯入畫圖的函式庫
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
#匯入處理影像的模組 opencv
import cv2

#匯入Python內建的Imaging Library
import PIL.Image

In [3]:
#匯入tensorflow
import tensorflow as tf


### 匯入vgg模型
#### https://www.tensorflow.org/api_docs/python/tf/keras/applications
#### (第一次執行，他會自己下載)
##### 顯示模型資訊: 模型名稱後面加上 .summary()

In [4]:
#讀入模型
vgg_model = tf.keras.applications.VGG19(include_top=True, weights='imagenet')

In [5]:
#試著顯示一下模型的資訊





In [6]:
#顯示layer的名稱
# for layer in vgg_model.layers:
#     print(layer.name)



In [7]:
#(補充)凍結權重讓他不訓練 
#vgg_model.trainable = False    #全部模型凍結

# for layer in vgg_model.layers[0:17]:     #凍結前四個Block
#     layer.trainable = False

### 影像前處理

In [8]:
#讀取影像 (以cv2為例)
img = cv2.imread('original-image_pk.png')
img = img[:,:,::-1]

### (1)影像尺寸

In [9]:
#顯示影像尺寸的方式：加上.shape
#img.shape

In [25]:
#vgg19輸入預設值為224 x 224
#將影像縮放成想要的尺寸
resize_img = tf.image.resize(img, (224, 224))

#也有別的工具
#resize_img=cv2.resize(img, (224, 224))

#補充:
#cv2作完會是整數(還是影像)，
#而tensorflow作完會是小數(更為精確)，更適合作深度學習，但要取整數才能畫出正確的圖。

In [11]:
#處理完可以用.shape確認尺寸
#resize_img.shape

### (2)影像數值

In [12]:
#影像數值是[0,255]，每個模型都有前處理的方式，將數值做rescale。
#直接使用對應的preprocess_input最快
normal_img = tf.keras.applications.vgg19.preprocess_input(resize_img)



https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg19/preprocess_input

補充：
使用VGG19的前處理
其數值預設採用mode = caffe的方式

即將影像**RGB**轉換為**BGR**,然後其數值減去ImageNet平均, 亦即減去 `BGR` [103.939, 116.779, 123.68]

故經此處理後 影像為**BGR**格式(<=重要, 需注意!!!)

其他常見的處理方式還有

mode = tf ( will scale pixels between -1 and 1, sample-wise. )

除以127.5，然後減 1

mode = torch
( will scale pixels between 0 and 1 and then
          will normalize each channel with respect to the
          ImageNet dataset.)

除以255，減去ImageNet平均[0.485, 0.456, 0.406] ，除以標準差[0.229, 0.224, 0.225]。

https://github.com/tensorflow/tensorflow/blob/85c8b2a817f95a3e979ecd1ed95bff1dc1335cff/tensorflow/python/keras/applications/imagenet_utils.py#L169

### (3)輸入格式

In [13]:
#要餵給AI時，它的輸入是batch, image_dim, channels
#batch = 1 一筆資料
#image_dim = 224, 224 影像尺寸
#channels = 3 (BGR)

input_img = tf.reshape(normal_img,  (1, 224, 224, 3), name=None)

In [14]:
#補充: 檢查format
tf.keras.backend.image_data_format()

'channels_last'

##### `channels_last` 代表輸入影像尺寸為(batch, image_dim, channels)
##### 而對應的`channels_first`輸入尺寸為(batch, channels, image_dim)
##### 若需要更改可使用 `tf.keras.backend.set_image_data_format`

## 用模型預測

In [15]:
#對影像進行預測，算出機率
#直接將輸入影像丟進模型裡
prediction_probabilities = vgg_model(input_img)

#如果用下面寫法，會的到一樣的結果，但格式為np_array
#prediction_probabilities = vgg_model.predict(input_img)

#利用np.argmax()可知道是哪個index的機率最大

In [16]:
prediction_probabilities.shape

TensorShape([1, 1000])

In [14]:
#利用官方的decode_predictions，可以還原出這張影像是對應Imagenet的哪個分類
predicted_top_5 = tf.keras.applications.vgg19.decode_predictions(prediction_probabilities.numpy())[0]
[(class_name, prob) for (number, class_name, prob) in predicted_top_5]

[('teddy', 0.23540515),
 ('Chihuahua', 0.10880504),
 ('toyshop', 0.092614815),
 ('bow_tie', 0.04598467),
 ('cowboy_hat', 0.033216592)]